In [3]:
import tensorflow as tf
from PIL import Image
tf.__version__

'2.0.0'

In [4]:
from mnist_decode import decode_idx3_ubyte,decode_idx1_ubyte

x_train = decode_idx3_ubyte('./input/coic/train-images-idx3-ubyte')
y_train = decode_idx1_ubyte('./input/coic/train-labels-idx1-ubyte')
print(x_train.shape,y_train.shape)
x_test = decode_idx3_ubyte('./input/coic/test-images-idx3-ubyte')
y_test = decode_idx1_ubyte('./input/coic/test-labels-idx1-ubyte')
print(x_test.shape,y_test.shape)

(18225, 28, 28) (18225,)
(4563, 28, 28) (4563,)


In [5]:
from keras.utils import np_utils
x_train = x_train.reshape(x_train.shape[0], 1, 28, 28).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 1, 28, 28).astype('float32')
x_train = x_train/255
x_test = x_test/255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(x_test.shape,y_test.shape)

(4563, 1, 28, 28) (4563, 10)


Using TensorFlow backend.


In [6]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds

<TensorSliceDataset shapes: ((1, 28, 28), (10,)), types: (tf.float32, tf.float32)>

In [7]:
train_ds = train_ds.shuffle(10000).batch(32)
# suffle表示隨機選取,並分為32batch
train_ds

<BatchDataset shapes: ((None, 1, 28, 28), (None, 10)), types: (tf.float32, tf.float32)>

In [8]:
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.shuffle(10000).batch(32)

In [9]:
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D, MaxPool2D
from tensorflow.keras.models import Model

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.c1= Conv2D(32,(3,3),padding='same',input_shape=(1,28,28))
        #self.c2= Conv1D(64,3,padding='same',activation='relu')
        self.mp1= MaxPool2D()
        #self.c3= Conv1D(64,3,padding='same',input_shape=(28,28))
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu') 
        self.d2 = Dense(128, activation='relu')
        self.d3 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.c1(x)
        x = self.mp1(x)
        #x = self.c2(x)
        #x = self.c3(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        return self.d3(x)

# Create an instance of the model
model = MyModel()


In [10]:

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


In [11]:
#@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    rain_accuracy(labels, predictions)

In [12]:
#@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

TypeError: build() missing 1 required positional argument: 'self'

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result()*100,
                          test_loss.result(),
                          test_accuracy.result()*100))

# Reset the metrics for the next epoch
train_loss.reset_states()
train_accuracy.reset_states()
test_loss.reset_states()
test_accuracy.reset_states()

In [ ]:
import mnist_decode
fina_test_x=decode_idx3_ubyte('./input/coic/final-test-images-idx3-ubyte')
model.preict(fina_test_x)